### Song Recommendation process

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import AdamW, get_cosine_schedule_with_warmup, BertTokenizer, BertModel
from collections import Counter
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer
import re
import io 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import logging
import gensim
from gensim.models import FastText
from sklearn.metrics.pairwise import cosine_similarity

Input necessary hyperparameters

In [2]:
bestnum = int(input("How many best? "))

diary_text = input("Input the diary contents: ")

seed = input("Input seed(num): ")

In [3]:
def del_bracket(s):
  pattern = r'\([^)]*\)'  # ()
  s = re.sub(pattern=pattern, repl='', string=s)

  pattern = r'\[[^)]*\]'  # []
  s = re.sub(pattern=pattern, repl='', string=s)

  pattern = r'\<[^)]*\>'  # <>
  s = re.sub(pattern=pattern, repl='', string=s)

  pattern = r'\{[^)]*\}'  # {}
  s = re.sub(pattern=pattern, repl='', string=s)

  return s

def del_special_num(s):
  pattern = r'[^a-zA-Z가-힣]'
  s = re.sub(pattern=pattern, repl=' ', string=s)

  return s

def del_unit(s):
  units = ['mm', 'cm', 'km', 'ml', 'kg', 'g']
  for unit in units:
    s = s.lower() # 대문자를 소문자로 변환
    s = s.replace(unit, '')
  return s

def del_whitespace(s):
  return " ".join(s.split())
  
def del_stopwords(s):
  stopwords = open("data/stopwords.txt", 'r', encoding="utf-8").read().split()
  #print(stopwords)
  s_o=s.split()
  s_f=[]
  for w in s_o:
    if w.strip() not in stopwords:
      s_f.append(w.strip())
  return " ".join(s_f)

Model 1-4 ongoing...

In [4]:
# importing model
modelname = "klue/bert-base"
model_path = 'models/fin_model_1.pt'  # replace with your actual path
max_length = 64
num_classes = 6

# Use cuda if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#BERT 모델 불러오기
bertmodel = BertModel.from_pretrained(modelname)

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=num_classes,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids, attention_mask):
        outputs = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        pooler = outputs[1]
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

# assuming you have a function for tokenization
tokenizer = AutoTokenizer.from_pretrained(modelname)

# Define a function to get the valid_length, attention_mask and segment_ids
def get_inputs(tokens):
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    valid_length = len(tokens)
    segment_ids = [0]*valid_length
    attention_mask = [1]*valid_length

    # Pad up to max length
    if valid_length < max_length:
        pad_length = max_length - valid_length
        tokens.extend(['[PAD]' for _ in range(pad_length)])
        attention_mask.extend([0]*pad_length)
        segment_ids.extend([0]*pad_length)

    # Convert tokens to IDs
    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    return torch.tensor([token_ids], dtype=torch.long), torch.tensor([valid_length], dtype=torch.long), torch.tensor([segment_ids], dtype=torch.long), torch.tensor([attention_mask], dtype=torch.long)

# assuming you have a function for pre-processing
def preprocess(text):
    for t in text:
        t=del_bracket(t)
        t=del_special_num(t)
        t=del_whitespace(t)
        t=del_stopwords(t)
    return text.lower()

def temperature_scaled_softmax(output, temperature=1.0):
    # Apply temperature scaling on logits
    output = output / temperature

    # Then apply softmax to convert to probabilities
    probabilities = F.softmax(output, dim=-1)

    return probabilities

# Load the model
if torch.cuda.is_available():
    model = torch.load(model_path)
elif not torch.cuda.is_available():
    model = torch.load(model_path, map_location=torch.device('cpu'))

# Switch to eval mode
model.eval()

# Load the model and move to the GPU if available
model.to(device)

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import ast
import json

def process_lyrics(lyrics, tokenizer, model):
    # Preprocess and tokenize the lyrics
    tokens = tokenizer.tokenize(preprocess(lyrics))
    if len(tokens) > max_length-2: # Account for [CLS] and [SEP]
        tokens = tokens[:max_length-2]

    # Get inputs
    token_ids, valid_length, segment_ids, attention_mask = get_inputs(tokens)

    # Move all your tensors to the same device as your model
    token_ids = token_ids.to(device)
    valid_length = valid_length.to(device)
    segment_ids = segment_ids.to(device)
    attention_mask = attention_mask.to(device)

    # Ensure no gradient is calculated
    with torch.no_grad():
        sentiment_vector = model(token_ids, valid_length, segment_ids, attention_mask)

    return sentiment_vector.detach().cpu().numpy()[0]  # Return as a 1-D numpy array

# Function to compute cosine similarities and retrieve the top 10 songs
def recommend_songs(diary_text):
    # Process the diary text
    diary_vector = process_lyrics(diary_text, tokenizer, model)

    # Compute cosine similarities
    similarities = cosine_similarity([diary_vector], df_music['sentiment_vector'].to_list())

    # Get the top 10 song indices
    Best_N = bestnum
    top_10_indices = similarities[0].argsort()[-Best_N:][::-1]

    # Return the corresponding songs
    return df_music.iloc[top_10_indices]

# Load the music data
df_music = pd.read_csv('library/music_library_model1.csv')

# Convert the strings back to arrays
df_music['sentiment_vector'] = df_music['sentiment_vector'].apply(lambda x: np.array(ast.literal_eval(x)))

recommend_songs(diary_text)[['title', 'artist']].to_csv(f"model1{seed}.csv", index=False, header=False)

cpu


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# importing model
modelname = "klue/bert-base" 
model_path = 'models/fin_model_2.pt'  # replace with your actual path
max_length = 64
num_classes = 7

# Use cuda if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#BERT 모델 불러오기
bertmodel = BertModel.from_pretrained(modelname)

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=num_classes,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids, attention_mask):
        outputs = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        pooler = outputs[1]
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

# assuming you have a function for tokenization
tokenizer = AutoTokenizer.from_pretrained(modelname)

# Define a function to get the valid_length, attention_mask and segment_ids
def get_inputs(tokens):
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    valid_length = len(tokens)
    segment_ids = [0]*valid_length
    attention_mask = [1]*valid_length

    # Pad up to max length
    if valid_length < max_length:
        pad_length = max_length - valid_length
        tokens.extend(['[PAD]' for _ in range(pad_length)])
        attention_mask.extend([0]*pad_length)
        segment_ids.extend([0]*pad_length)

    # Convert tokens to IDs
    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    return torch.tensor([token_ids], dtype=torch.long), torch.tensor([valid_length], dtype=torch.long), torch.tensor([segment_ids], dtype=torch.long), torch.tensor([attention_mask], dtype=torch.long)

# assuming you have a function for pre-processing
def preprocess(text):
    for t in text:
        t=del_bracket(t)
        t=del_special_num(t)
        t=del_whitespace(t)
        t=del_stopwords(t)
    return text.lower()

def temperature_scaled_softmax(output, temperature=1.0):
    # Apply temperature scaling on logits
    output = output / temperature

    # Then apply softmax to convert to probabilities
    probabilities = F.softmax(output, dim=-1)

    return probabilities

# Load the model
if torch.cuda.is_available():
    model = torch.load(model_path)
elif not torch.cuda.is_available():
    model = torch.load(model_path, map_location=torch.device('cpu'))

# Switch to eval mode
model.eval()

# Load the model and move to the GPU if available
model.to(device)

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import ast
import json

def process_lyrics(lyrics, tokenizer, model):
    # Preprocess and tokenize the lyrics
    tokens = tokenizer.tokenize(preprocess(lyrics))
    if len(tokens) > max_length-2: # Account for [CLS] and [SEP]
        tokens = tokens[:max_length-2]

    # Get inputs
    token_ids, valid_length, segment_ids, attention_mask = get_inputs(tokens)

    # Move all your tensors to the same device as your model
    token_ids = token_ids.to(device)
    valid_length = valid_length.to(device)
    segment_ids = segment_ids.to(device)
    attention_mask = attention_mask.to(device)

    # Ensure no gradient is calculated
    with torch.no_grad():
        sentiment_vector = model(token_ids, valid_length, segment_ids, attention_mask)

    return sentiment_vector.detach().cpu().numpy()[0]  # Return as a 1-D numpy array

# Function to compute cosine similarities and retrieve the top 10 songs
def recommend_songs(diary_text):
    # Process the diary text
    diary_vector = process_lyrics(diary_text, tokenizer, model)

    # Compute cosine similarities
    similarities = cosine_similarity([diary_vector], df_music['sentiment_vector'].to_list())

    # Get the top 10 song indices
    Best_N = bestnum
    top_10_indices = similarities[0].argsort()[-Best_N:][::-1]

    # Return the corresponding songs
    return df_music.iloc[top_10_indices]

# Function to print out the recommended songs
def print_recommended_songs(diary_text):
    recommended_songs = recommend_songs(diary_text)
    print("Top 10 similar songs:\n")
    print("Rank\tSimilarity\tSong Name - Artist")
    for i, song in enumerate(recommended_songs.iterrows(), start=1):
        index, data = song
        similarity = cosine_similarity([process_lyrics(diary_text, tokenizer, model)], [data['sentiment_vector']])[0][0]
        print(f"{i}st\t{similarity*100:.2f}% similar\t{data['title']} - {data['artist']}")

# Load the music data
df_music = pd.read_csv('library/music_library_model2.csv')

# Convert the strings back to arrays
df_music['sentiment_vector'] = df_music['sentiment_vector'].apply(lambda x: np.array(ast.literal_eval(x)))
recommend_songs(diary_text)[['title', 'artist']].to_csv(f"model2{seed}.csv", index=False, header=False)

cpu


Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# importing model
modelname = "kykim/bert-kor-base" 
model_path = 'models/fin_model_3.pt'  # replace with your actual path
max_length = 64
num_classes = 9

# Use cuda if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#BERT 모델 불러오기
bertmodel = BertModel.from_pretrained(modelname)

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=num_classes,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids, attention_mask):
        outputs = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        pooler = outputs[1]
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

# assuming you have a function for tokenization
tokenizer = AutoTokenizer.from_pretrained(modelname)

# Define a function to get the valid_length, attention_mask and segment_ids
def get_inputs(tokens):
    tokens = ['[CLS]'] + tokens + ['[SEP]']
    valid_length = len(tokens)
    segment_ids = [0]*valid_length
    attention_mask = [1]*valid_length

    # Pad up to max length
    if valid_length < max_length:
        pad_length = max_length - valid_length
        tokens.extend(['[PAD]' for _ in range(pad_length)])
        attention_mask.extend([0]*pad_length)
        segment_ids.extend([0]*pad_length)

    # Convert tokens to IDs
    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    return torch.tensor([token_ids], dtype=torch.long), torch.tensor([valid_length], dtype=torch.long), torch.tensor([segment_ids], dtype=torch.long), torch.tensor([attention_mask], dtype=torch.long)

# assuming you have a function for pre-processing
def preprocess(text):
    for t in text:
        t=del_bracket(t)
        t=del_special_num(t)
        t=del_whitespace(t)
        t=del_stopwords(t)
    return text.lower()

def temperature_scaled_softmax(output, temperature=1.0):
    # Apply temperature scaling on logits
    output = output / temperature

    # Then apply softmax to convert to probabilities
    probabilities = F.softmax(output, dim=-1)

    return probabilities

# Load the model
if torch.cuda.is_available():
    model = torch.load(model_path)
elif not torch.cuda.is_available():
    model = torch.load(model_path, map_location=torch.device('cpu'))

# Switch to eval mode
model.eval()

# Load the model and move to the GPU if available
model.to(device)

import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import ast
import json

def process_lyrics(lyrics, tokenizer, model):
    # Preprocess and tokenize the lyrics
    tokens = tokenizer.tokenize(preprocess(lyrics))
    if len(tokens) > max_length-2: # Account for [CLS] and [SEP]
        tokens = tokens[:max_length-2]

    # Get inputs
    token_ids, valid_length, segment_ids, attention_mask = get_inputs(tokens)

    # Move all your tensors to the same device as your model
    token_ids = token_ids.to(device)
    valid_length = valid_length.to(device)
    segment_ids = segment_ids.to(device)
    attention_mask = attention_mask.to(device)

    # Ensure no gradient is calculated
    with torch.no_grad():
        sentiment_vector = model(token_ids, valid_length, segment_ids, attention_mask)

    return sentiment_vector.detach().cpu().numpy()[0]  # Return as a 1-D numpy array

# Function to compute cosine similarities and retrieve the top 10 songs
def recommend_songs(diary_text):
    # Process the diary text
    diary_vector = process_lyrics(diary_text, tokenizer, model)

    # Compute cosine similarities
    similarities = cosine_similarity([diary_vector], df_music['sentiment_vector'].to_list())

    # Get the top 10 song indices
    Best_N = bestnum
    top_10_indices = similarities[0].argsort()[-Best_N:][::-1]

    # Return the corresponding songs
    return df_music.iloc[top_10_indices]

# Function to print out the recommended songs
def print_recommended_songs(diary_text):
    recommended_songs = recommend_songs(diary_text)
    print("Top 10 similar songs:\n")
    print("Rank\tSimilarity\tSong Name - Artist")
    for i, song in enumerate(recommended_songs.iterrows(), start=1):
        index, data = song
        similarity = cosine_similarity([process_lyrics(diary_text, tokenizer, model)], [data['sentiment_vector']])[0][0]
        print(f"{i}st\t{similarity*100:.2f}% similar\t{data['title']} - {data['artist']}")

# Load the music data
df_music = pd.read_csv('library/music_library_model3.csv')

# Convert the strings back to arrays
df_music['sentiment_vector'] = df_music['sentiment_vector'].apply(lambda x: np.array(ast.literal_eval(x)))

recommend_songs(diary_text)[['title', 'artist']].to_csv(f"model3{seed}.csv", index=False, header=False)

cpu


Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
# Load the saved FastText model
loaded_fasttext_model = FastText.load("data/trained_fasttext_model")

# Function to compute average FastText vector for a text
def text_to_avg_vector(text, model):
    words = text.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv.key_to_index]
    
    if not word_vectors:
        return None
    
    return sum(word_vectors) / len(word_vectors)

# Load lyrics from the CSV file
lyrics_df = pd.read_csv('library/music_library.csv')

# Compute average FastText vectors for lyrics and store them with song identifier
song_vectors = {}
for index, row in lyrics_df.iterrows():
    lyrics_vector = text_to_avg_vector(row['lyrics'], loaded_fasttext_model)
    if lyrics_vector is not None:
        song_identifier = row['title'] + ' - ' + row['artist']
        song_vectors[song_identifier] = lyrics_vector

# Make system to get manual input
diary_entry = diary_text

# Compute the average FastText vector for the diary entry
diary_vector = text_to_avg_vector(diary_entry, loaded_fasttext_model)

# Compute cosine similarity between diary entry and song lyrics
similarities = {}
for song_identifier, lyrics_vector in song_vectors.items():
    similarity = cosine_similarity(diary_vector.reshape(1, -1), lyrics_vector.reshape(1, -1))
    similarities[song_identifier] = similarity[0][0]

# Rank songs based on similarity and recommend top N songs
N = bestnum
# N = 5
recommended_songs = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:N]

songinfo = [sublist[0] for sublist in recommended_songs]

# Split each string in the list into a "songname" and an "artist"
songs_split = [song.split(' - ') for song in songinfo]

# Convert the list to a pandas DataFrame
df = pd.DataFrame(songs_split)

# Specify the file path (or name if the file is in the same directory as your script)
file_path = f'model4{seed}.csv'

# Write the DataFrame to a CSV file without header and index
df.to_csv(file_path, index=False, header=False)

### Match & Shuffle process

In [8]:
import pandas as pd
from sklearn.utils import shuffle

seed = input("Input seed(num): ")

# Load the songs from the CSV files
model1 = pd.read_csv(f'model1{seed}.csv', header=None)
model1['model'] = 'model1'

model2 = pd.read_csv(f'model2{seed}.csv', header=None)
model2['model'] = 'model2'

model3 = pd.read_csv(f'model3{seed}.csv', header=None)
model3['model'] = 'model3'

model4 = pd.read_csv(f'model4{seed}.csv', header=None)
model4['model'] = 'model4'

# Concatenate into a single DataFrame
all_songs = pd.concat([model1, model2, model3, model4], ignore_index=True)

# Shuffle the songs
all_songs = shuffle(all_songs, random_state=1)

# Assign each song a unique ID
all_songs['id'] = range(1, len(all_songs) + 1)

# Now, all_songs is your list of 40 songs, with randomly ordered with id of 01 to 40

def evaluate_selection(selection):
    # Find the model each selected song came from
    selected_models = all_songs[all_songs['id'].isin(selection)]['model']
    
    # Count how many songs from each model were selected
    model_counts = selected_models.groupby(selected_models).size()
    
    # Reindex to specify the order and include models with zero count
    model_counts = model_counts.reindex(['model1', 'model2', 'model3', 'model4'], fill_value=0)

    # Total number of selections
    total_selections = len(selection)
    
    # Prepare the output string
    output = f"For your selection, {selection}\nFollowing scored....\n"
    
    max_model = ""
    max_count = 0
    for model, count in model_counts.items():
        percentage = (count / total_selections) * 100
        output += f"{model.capitalize()}: {count} ({percentage:.2f}%)\n"

        if count > max_count:
            max_model = model
            max_count = count

    output += f"{max_model.capitalize()} will be the best."
    
    return output

In [9]:
# Display the songs and their IDs
print('#'*120)
print('#'*50 + " Shuffled song list " + '#'*50)
print('#'*120)
print(all_songs[['id', 0, 1]].to_string(index=False))
print('#'*80)

try:
    selection = input("Input best 10: ").split(',')
    selection = [int(s) for s in selection]
    print("Your Selection(id) is:",selection)
    print("#"*80)
    
    # Test the function with a selection
    print(evaluate_selection(selection))
except:
    print("canceled")

################################################################################
############################## Shuffled song list ##############################
################################################################################
 id                                                    0                                          1
  1                      내게만 일어나는 일 (Feat. MC 메타 Of 가리온)                                        이승환
  2              사랑엔 조건이 없습니다 (법무부 'Good-Bye 학교폭력' 캠페인송) 부활, 걸스데이, 배기성, 백청강, 손진영, 이성욱, 이태권, 정단, 최재훈
  3 내가 너의 오아시스가 되어줄께 5 (괴물소년) (Feat. HEX Of Vanila CIty)                                         팻두
  4                                       지난날(Feat.재주소년)                                        김윤희
  5                                              나의 매일에게                                     안녕하신가영
  6                                              27살의 고백                                        윤딴딴
  7                          BENZ (Feat. Jibin of Y2K92) 